## 부동산 실거래가 Crawling ㄱㄱ

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import re


def get_seoul_real_estate_data():
    # API 파라미터 설정
    # 1 접수연도
    RCPT_YR = 2025          # 현재 2025년 데이터 -> 연도 변환 하면 됨
    var1 = RCPT_YR

    # 2 자치구코드
    CGG_CD = '%20'
    var2 = CGG_CD

    # 3 자치구명
    CGG_NM = '강남구'       # 서울시 행정구 입력
    var3 = CGG_NM

    # 4 법정동코드
    STDG_CD = '%20'
    var4 = STDG_CD

    # 5 지번구분
    LOTNO_SE = '%20'
    var5 = LOTNO_SE

    # 6 지번구분명
    LOTNO_SE_NM = '%20'
    var6 = LOTNO_SE_NM

    # 7 본번
    MNO = '%20'
    var7 = MNO

    # 8 부번
    SNO = '%20'
    var8 = SNO

    # 9 건물명
    BLDG_NM = '%20'
    var9 = BLDG_NM

    # 10 계약일
    CTRT_DAY = '%20'
    var10 = CTRT_DAY

    # 11 건물용도 			# [아파트/단독다가구/연립다세대/오피스텔] 택 1
    BLDG_USG = '아파트' 
    var11 = BLDG_USG

    vKey = '646e476d7a62757235397547714b41'
    base_url = f'http://openapi.seoul.go.kr:8088/{vKey}/xml/tbLnOpendataRtmsV'
    
    try:
        # 1. 전체 데이터 개수 확인
        count_url = f'{base_url}/1/1/{var1}/{var2}/{var3}/{var4}/{var5}/{var6}/{var7}/{var8}/{var9}/{var10}/{var11}'
        count_response = requests.get(count_url)
        root = ET.fromstring(count_response.content)
        
        # 에러 체크
        result = root.find('RESULT')
        if result is not None:
            code = result.find('CODE')
            if code is not None and code.text != 'INFO-000':
                message = result.find('MESSAGE')
                print(f"🚨 API 오류: {code.text} - {message.text if message is not None else '알 수 없는 오류'}")
                return None
        
        total_count = int(root.find('list_total_count').text)
        print(f"📌 전체 데이터 개수: {total_count}건")
        
        # 2. 1000건씩 나누어서 데이터 가져오기
        all_data = []
        max_per_request = 1000  # API 제한사항
        
        for start_idx in range(1, total_count + 1, max_per_request):
            end_idx = min(start_idx + max_per_request - 1, total_count)
            
            print(f"📥 데이터 수집 중: {start_idx}~{end_idx}번째 ({len(range(start_idx, end_idx + 1))}건)")
            
            data_url = f'{base_url}/{start_idx}/{end_idx}/{var1}/{var2}/{var3}/{var4}/{var5}/{var6}/{var7}/{var8}/{var9}/{var10}/{var11}'
            
            # 3. 각 배치 데이터 요청
            response = requests.get(data_url, timeout=30)
            response.raise_for_status()
            
            # XML 파싱하여 데이터 추출
            batch_root = ET.fromstring(response.content)
            
            # 에러 체크
            batch_result = batch_root.find('RESULT')
            if batch_result is not None:
                batch_code = batch_result.find('CODE')
                if batch_code is not None and batch_code.text != 'INFO-000':
                    batch_message = batch_result.find('MESSAGE')
                    print(f"⚠️ 배치 {start_idx}~{end_idx} 오류: {batch_code.text} - {batch_message.text if batch_message is not None else '알 수 없는 오류'}")
                    continue
            
            # row 데이터 추출
            for row in batch_root.findall('row'):
                row_data = {}
                for element in row:
                    tag_name = element.tag
                    tag_value = element.text if element.text else ''
                    row_data[tag_name] = tag_value
                all_data.append(row_data)
            
            # API 호출 간격 (서버 부하 방지)
            import time
            time.sleep(0.1)
        
        print(f"✅ 총 수집된 데이터: {len(all_data)}건")
        
        # 4. 전체 데이터를 XML 형태로 재구성
        combined_xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<response>
    <list_total_count>{total_count}</list_total_count>
    <RESULT>
        <CODE>INFO-000</CODE>
        <MESSAGE>정상 처리되었습니다</MESSAGE>
    </RESULT>
"""
        
        for data in all_data:
            combined_xml += "    <row>\n"
            for key, value in data.items():
                combined_xml += f"        <{key}>{value}</{key}>\n"
            combined_xml += "    </row>\n"
        
        combined_xml += "</response>"
        
        return combined_xml.encode('utf-8')

    except requests.exceptions.RequestException as e:
        print(f"🚨 API 호출 오류: {e}")
        return None
    except ET.ParseError as e:
        print(f"🚨 XML 파싱 오류: {e}")
        return None
    except Exception as e:
        print(f"🚨 예상치 못한 오류: {e}")
        return None


def parse_xml_to_dataframe(xml_content):
    """XML 데이터를 DataFrame으로 변환"""
    if xml_content is None:
        return pd.DataFrame()
    
    try:
        # XML 파싱
        root = ET.fromstring(xml_content)
        
        # 전체 데이터 개수 확인
        total_count = root.find('list_total_count')
        if total_count is not None:
            print(f"총 데이터 개수: {total_count.text}")
        
        # 결과 상태 확인
        result = root.find('RESULT')
        if result is not None:
            code = result.find('CODE')
            message = result.find('MESSAGE')
            if code is not None and message is not None:
                print(f"결과 코드: {code.text}, 메시지: {message.text}")
        
        # 데이터 리스트 초기화
        data_list = []
        
        # 각 row 요소 처리
        for row in root.findall('row'):
            row_data = {}
            
            # 모든 하위 요소 추출
            for element in row:
                tag_name = element.tag
                tag_value = element.text if element.text else ''
                row_data[tag_name] = tag_value
            
            data_list.append(row_data)
        
        print(f"파싱된 데이터 행 수: {len(data_list)}")
        
        # DataFrame 생성
        df = pd.DataFrame(data_list)
        
        return df
    
    except ET.ParseError as e:
        print(f"XML 파싱 오류: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"데이터 처리 오류: {e}")
        return pd.DataFrame()


def preprocess_dataframe(df):
    """DataFrame 전처리 및 추가 컬럼 생성"""
    if df.empty:
        return df
    
    # 숫자형 데이터 변환
    numeric_columns = ['RCPT_YR', 'CGG_CD', 'STDG_CD', 'MNO', 'SNO', 
                      'THING_AMT', 'ARCH_AREA', 'LAND_AREA', 'FLR', 'ARCH_YR']
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 날짜 변환
    date_columns = ['CTRT_DAY', 'RTRCN_DAY']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce')
    
    # 건축연대 그룹화
    if 'ARCH_YR' in df.columns:
        df['ARCH_DECADE'] = df['ARCH_YR'].apply(
            lambda x: f"{(x//10)*10}년대" if pd.notna(x) else "미상"
        )
    
    # 평수 계산 (건축면적 기준)
    if 'ARCH_AREA' in df.columns:
        df['PYEONG'] = df['ARCH_AREA'] * 0.3025  # ㎡ → 평 변환
        df['PYEONG_GROUP'] = df['PYEONG'].apply(
            lambda x: f"{int(x//10)*10}평형대" if pd.notna(x) and x > 0 else "미상"
        )
    
    # 거래금액을 억원 단위로 변환
    if 'THING_AMT' in df.columns:
        df['PRICE_EUK'] = df['THING_AMT'] / 10000
    
    # 데이터 정렬 (최신 계약일 순)
    if 'CTRT_DAY' in df.columns:
        df = df.sort_values('CTRT_DAY', ascending=False)
    
    return df


def save_to_csv(df, filename=None):
    """DataFrame을 CSV 파일로 저장"""
    if df.empty:
        print("저장할 데이터가 없습니다.")
        return
    
    # data 디렉토리 생성
    import os
    os.makedirs('data', exist_ok=True)
    
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'data/{timestamp}_seoul_real_estate.csv'
    
    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ 데이터 저장 완료: {filename}")
        print(f"📊 총 {len(df)}건의 데이터")
        
        # 기본 통계 출력
        if 'THING_AMT' in df.columns:
            print(f"💰 평균 거래가: {df['THING_AMT'].mean():,.0f}만원")
            print(f"💰 최고 거래가: {df['THING_AMT'].max():,.0f}만원")
        
        if 'ARCH_DECADE' in df.columns:
            print("\n🏢 건축연대별 분포:")
            print(df['ARCH_DECADE'].value_counts().head())
        
        if 'PYEONG_GROUP' in df.columns:
            print("\n📐 평수대별 분포:")
            print(df['PYEONG_GROUP'].value_counts().head())
            
    except Exception as e:
        print(f"파일 저장 오류: {e}")



def main():
    """메인 실행 함수"""
    print("🚀 서울시 부동산 실거래가 데이터 수집 시작")
    print("=" * 50)
    
    # 1. XML 데이터 수집
    xml_content = get_seoul_real_estate_data()
    
    if xml_content is None:
        print("❌ 데이터 수집 실패")
        return
    
    # 2. XML 파싱 및 DataFrame 변환
    print("\n📊 XML 데이터 파싱 중...")
    df = parse_xml_to_dataframe(xml_content)
    
    if df.empty:
        print("❌ 파싱된 데이터가 없습니다.")
        return
    
    # 3. 데이터 전처리
    print("\n🔧 데이터 전처리 중...")
    df = preprocess_dataframe(df)
    
    # 4. DataFrame 정보 출력
    print("\n📋 DataFrame 정보:")
    print(f"행 수: {len(df)}")
    print(f"열 수: {len(df.columns)}")
    print("\n컬럼 목록:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")
    
    # 5. 샘플 데이터 출력
    print("\n🔍 샘플 데이터 (상위 3개):")
    print(df.head(3)[['CGG_NM', 'STDG_NM', 'BLDG_NM', 'CTRT_DAY', 'THING_AMT', 'ARCH_AREA']].to_string())
    
    # 6. CSV 저장
    print("\n💾 CSV 파일 저장 중...")
    save_to_csv(df)
    
    print("\n✅ 작업 완료!")
    
    return df


# 실행
if __name__ == "__main__":
    df = main()


🚀 서울시 부동산 실거래가 데이터 수집 시작
📌 전체 데이터 개수: 1959건
📥 데이터 수집 중: 1~1000번째 (1000건)
📥 데이터 수집 중: 1001~1959번째 (959건)
✅ 총 수집된 데이터: 1959건

📊 XML 데이터 파싱 중...
총 데이터 개수: 1959
결과 코드: INFO-000, 메시지: 정상 처리되었습니다
파싱된 데이터 행 수: 1959

🔧 데이터 전처리 중...

📋 DataFrame 정보:
행 수: 1959
열 수: 25

컬럼 목록:
 1. RCPT_YR
 2. CGG_CD
 3. CGG_NM
 4. STDG_CD
 5. STDG_NM
 6. LOTNO_SE
 7. LOTNO_SE_NM
 8. MNO
 9. SNO
10. BLDG_NM
11. CTRT_DAY
12. THING_AMT
13. ARCH_AREA
14. LAND_AREA
15. FLR
16. RGHT_SE
17. RTRCN_DAY
18. ARCH_YR
19. BLDG_USG
20. DCLR_SE
21. OPBIZ_RESTAGNT_SGG_NM
22. ARCH_DECADE
23. PYEONG
24. PYEONG_GROUP
25. PRICE_EUK

🔍 샘플 데이터 (상위 3개):
  CGG_NM STDG_NM                  BLDG_NM   CTRT_DAY  THING_AMT  ARCH_AREA
0    강남구    압구정동  현대14차(203,204,205,206동) 2025-06-02     520000      84.98
1    강남구     개포동                개포래미안포레스트 2025-06-02     320000      84.90
2    강남구     자곡동                  래미안강남힐즈 2025-05-30     215000     101.94

💾 CSV 파일 저장 중...
✅ 데이터 저장 완료: data/20250604_163131_seoul_real_estate.csv
📊 총 1959건의 데이터


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime
import re
import requests

driver = webdriver.Chrome()
driver.maximize_window()

#response = requests.get(url)
#print(response.content)

In [23]:
vKey = '646e476d7a62757235397547714b41'
url = f'http://openapi.seoul.go.kr:8088/{vKey}/xml/tbLnOpendataRtmsV/1/5/{var1}/{var2}/{var3}/{var4}/{var5}/{var6}/{var7}/{var8}/{var9}/{var10}/{var11}'

print(url)

driver.get(url)

http://openapi.seoul.go.kr:8088/646e476d7a62757235397547714b41/xml/tbLnOpendataRtmsV/1/5/2025/11500/강서구/10500/%20/%20/%20/%20/%20/%20/아파트
